In [3]:
from importlib import reload
import pandas as pd
import numpy as np
from datetime import datetime
import datatools
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import pvlib
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import pvanalytics
import seaborn

%matplotlib inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
mpl.rcParams['font.weight']='bold'
mpl.rcParams['axes.titlesize']=22

In [2]:
def sys_met_data(mettablename, systablename, start, end):
    engine = datatools.database.create_mss_engine()
    
    met_sql = f"select * from {mettablename} where TmStamp between '{start}' and '{end}';"
    met = pd.read_sql(met_sql, engine, index_col='TmStamp')
    met.index = pd.DatetimeIndex(met.index)
    met.index = met.index.tz_localize('MST') 
    
    sys_sql = f"select * from {systablename} where TmStamp between '{start}' and '{end}';"
    sys = pd.read_sql(sys_sql, engine, index_col='TmStamp')
    sys.index = pd.DatetimeIndex(sys.index)
    sys.index = sys.index.tz_localize('MST')

    df = pd.merge(met,sys,how='inner', left_index=True, right_index=True)
    
    return df

In [2]:
def calc_sol_data(time, latitude, longitude, tilt,altitude, temperature):
    sdf = pvlib.solarposition.get_solarposition(time, latitude, longitude,altitude, temperature)
    sdf['dni_extra'] = pvlib.irradiance.get_extra_radiation(time)
    return sdf

In [4]:
def  calc_env_data(tilt, altitude, sur_azimuth, sol_azimuth, zenith, DNI, GHI, DHI, DNI_extra, model):
    edf = pvlib.irradiance.get_total_irradiance(tilt, sur_azimuth, zenith, sol_azimuth,
                                                               DNI, GHI, DHI, DNI_extra, model)
    edf['aoi'] = pvlib.irradiance.aoi(tilt, sur_azimuth, zenith, sol_azimuth)
    edf['airmass'] = pvlib.atmosphere.get_relative_airmass(zenith)
    return edf

In [5]:
def meas_val(str_v, str_i):
    str_p = str_v*str_i
    results = str_p
    return results

In [6]:
def sapm_param(POA_global, POA_direct, POA_diffuse, amb_temp, ws_avg, am_abs, aoi, module):
    temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
    sapm_data = pvlib.temperature.sapm_cell(POA_global,
            amb_temp, ws_avg,
            **temperature_model_parameters)
    sapm_data = sapm_data.to_frame()
    sapm_data = sapm_data.rename(columns = {0 : 'tcell'})
    sapm_data['eff_irr'] = pvlib.pvsystem.sapm_effective_irradiance(
           POA_direct, POA_diffuse, am_abs, aoi, module)
    return sapm_data

In [7]:
def sapm(eff_irr, tcell, module):
    dc = pvlib.pvsystem.sapm(eff_irr, tcell, module)
    return dc

In [8]:
def sapm_p(v_mp, i_mp, str_len):
    str_v = v_mp*str_len
    str_p = i_mp*str_v
    return str_p

In [9]:
def pvwatts(POA_eff, cell_temp, stc_mod_p, Gpmp, temp_ref, str_len):
    p = pvlib.pvsystem.pvwatts_dc(POA_eff, cell_temp, stc_mod_p, Gpmp, temp_ref)
    str_p = p*str_len
    return str_p

In [10]:
def sdm(p_mp, str_len):
    str_p = p_mp*str_len
    return str_p

In [ ]:
def visual_dots(sys, mask, ylim_lower, ylim_upper):

    masked_sys = sys[~mask]
    plt.scatter(sys.index, sys, c='#add8e6',s=6, label='data', alpha=0.4)
    plt.scatter(masked_sys.index, masked_sys, c='#da9b86', label='mask')
    plt.xlabel(sys.index.name)
    plt.ylabel(sys.name)
    plt.ylim(ylim_lower, ylim_upper)
    #plt.xlim(xlim_lower, xlim_upper)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.show()
    return

In [ ]:
 def simple_filter(sys, variable, lower=None, upper=None, apply=False):

    variable = sys[variable]

# Physical Limits  (note: mask = True for data in acceptable ranges)


    if lower==None:
        mask_low = pd.Series(dtype='float64')
        if upper==None:
            return print('Please specify upper and/or lower limits')
        else:
            mask_high = variable<=upper
            mask = mask_high
    else:
        mask_low = variable >= lower
        if upper==None:
            mask_high = pd.Series(dtype='float64')
            mask = mask_low
        else:
            mask_high = variable<= upper
            mask = mask_low & mask_high
        
### Should add a check if the values are NaNs?  
    mask.name = f'mask_{variable.name}'

    if apply==True:
        sys = sys.where(mask, np.nan)
        return sys
    else:
        return mask